## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,Srednekolymsk,67.45,153.68,-26.46,86,54,1.34,broken clouds,RU
1,Aklavik,68.22,-135.01,-0.40,84,5,2.24,clear sky,CA
2,Weiz,47.22,15.62,34.00,99,100,3.51,overcast clouds,AT
3,College,64.86,-147.80,-7.01,84,100,1.30,overcast clouds,US
4,Albany,42.60,-73.97,41.00,63,100,3.13,overcast clouds,US


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))


What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                      (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
6,New Norfolk,-42.78,147.06,62.01,71,100,1.01,light rain,AU
27,Port Alfred,-33.59,26.89,69.08,67,99,22.44,overcast clouds,ZA
29,Mount Gambier,-37.83,140.77,59.00,82,75,19.46,broken clouds,AU
41,Hobart,-42.88,147.33,62.01,72,75,6.93,light rain,AU
43,Codrington,-38.27,141.97,65.88,56,0,20.96,clear sky,AU
44,Broken Hill,-31.95,141.43,73.40,69,75,16.11,broken clouds,AU
48,Luderitz,-26.65,15.16,66.45,61,21,15.88,few clouds,NaN
50,Kruisfontein,-34.00,24.73,66.99,71,97,4.00,overcast clouds,ZA
58,Naze,28.37,129.48,68.00,88,75,18.34,light intensity shower rain,JP
67,Hermanus,-34.42,19.23,70.00,59,20,11.01,few clouds,ZA


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   160
Lat                    160
Lng                    160
Max Temp               160
Humidity               160
Cloudiness             160
Wind Speed             160
Current Description    160
Country                158
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City                   158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
Country                158
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,New Norfolk,AU,62.01,light rain,-42.78,147.06,
27,Port Alfred,ZA,69.08,overcast clouds,-33.59,26.89,
29,Mount Gambier,AU,59.00,broken clouds,-37.83,140.77,
41,Hobart,AU,62.01,light rain,-42.88,147.33,
43,Codrington,AU,65.88,clear sky,-38.27,141.97,
44,Broken Hill,AU,73.40,broken clouds,-31.95,141.43,
50,Kruisfontein,ZA,66.99,overcast clouds,-34.00,24.73,
58,Naze,JP,68.00,light intensity shower rain,28.37,129.48,
67,Hermanus,ZA,70.00,few clouds,-34.42,19.23,
68,Ixtapa,MX,71.01,overcast clouds,20.70,-105.20,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   158
Country                158
Max Temp               158
Current Description    158
Lat                    158
Lng                    158
Hotel Name             158
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))